Import Libraries and variables

In [57]:
import pandas as pd

df=pd.read_csv("./datasets/all_vehicles.csv")

noise_words=["brand new","b/n","brand n","brandnew","new","used","1st","1st owner","petrol","diesel","hybrid","electric","Anniversary",
"second","2nd owner","company maintained","first","owner","owners","manual","auto","automatic","m/t","a/t","mt",
"full option","full options","full-option","full-options","full optioned","full-option","highest spec","high spec","full spec","Pre Order","Moon Roof",
"moonroof","(brandnew)","(brand new)","(moonroof)"]

df.head()

,Manufacturer,Model,Year,Fuel Type,Price,Mileage,Average Price (0-20k km),Average Price (20k-50k km),Average Price (50k-100k km),Average Price (100k+ km),Condition
0,Audi,Q7 Platinum Edition,2025.0,Petrol,73000000,0,73000000.0,NaN,NaN,NaN,NaN
1,Audi,Q7 Quattro,2025.0,Petrol,75000000,50,75000000.0,NaN,35825000.0,25250000.0,NaN
2,Audi,A1 TFSI 25,2024.0,Petrol,15450000,3350,15650000.0,NaN,NaN,NaN,NaN
3,Audi,A3,2024.0,Petrol,24500000,7000,24000000.0,NaN,13500000.0,NaN,NaN
4,Audi,A3 S Line Highest Spec,2024.0,Petrol,25000000,300,25000000.0,NaN,NaN,NaN,NaN


Adding columns

In [58]:
#adding body type column
if "Body Type" not in df.columns:
    col_index=df.columns.get_loc("Model")+1
    df.insert(col_index,"Body Type",None)
else:
    print("Body Type column already exists")

#adding seating capacity column
if "Seating Capacity" not in df.columns:
    col_index=df.columns.get_loc("Fuel Type")+1
    df.insert(col_index,"Seating Capacity",None)
else:
    print("Seating Capacity column already exists")

#adding ground clearance column
if "Ground Clearance" not in df.columns:
    col_index=df.columns.get_loc("Seating Capacity")+1
    df.insert(col_index,"Ground Clearance",None)
else:
    print("Ground Clearance column already exists")

df.head()



,Manufacturer,Model,Body Type,Year,Fuel Type,Seating Capacity,Ground Clearance,Price,Mileage,Average Price (0-20k km),Average Price (20k-50k km),Average Price (50k-100k km),Average Price (100k+ km),Condition
0,Audi,Q7 Platinum Edition,None,2025.0,Petrol,None,None,73000000,0,73000000.0,NaN,NaN,NaN,NaN
1,Audi,Q7 Quattro,None,2025.0,Petrol,None,None,75000000,50,75000000.0,NaN,35825000.0,25250000.0,NaN
2,Audi,A1 TFSI 25,None,2024.0,Petrol,None,None,15450000,3350,15650000.0,NaN,NaN,NaN,NaN
3,Audi,A3,None,2024.0,Petrol,None,None,24500000,7000,24000000.0,NaN,13500000.0,NaN,NaN
4,Audi,A3 S Line Highest Spec,None,2024.0,Petrol,None,None,25000000,300,25000000.0,NaN,NaN,NaN,NaN


Normalization

In [59]:
#Lowercase and remove white spaces from the end and beginning
df["Model"]=df["Model"].str.lower().str.strip()

#Shorting
df=df.sort_values(by=["Manufacturer","Model","Year"],ascending=[True,True,False])

#remove noise words
for word in noise_words:
    df["Model"]=df["Model"].str.replace(word,"",regex=True)
df["Model"]=df["Model"].str.replace(r"\s+"," ",regex=True).str.strip()

df.head()

,Manufacturer,Model,Body Type,Year,Fuel Type,Seating Capacity,Ground Clearance,Price,Mileage,Average Price (0-20k km),Average Price (20k-50k km),Average Price (50k-100k km),Average Price (100k+ km),Condition
45,Audi,a1,None,2018.0,Petrol,None,None,9600000,72000,NaN,NaN,9200000.0,NaN,NaN
66,Audi,a1,None,2017.0,Petrol,None,None,8800000,91600,NaN,NaN,9200000.0,NaN,NaN
75,Audi,a1,None,2016.0,Petrol,None,None,9350000,87000,NaN,NaN,9350000.0,NaN,NaN
46,Audi,a1 s-line rs1,None,2018.0,Petrol,None,None,10890000,27000,NaN,10890000.0,NaN,NaN,NaN
76,Audi,a1 tfsi,None,2016.0,Petrol,None,None,9700000,43000,NaN,9700000.0,NaN,NaN,NaN


Body type

In [60]:
###Body Type
audi_series_bt={"a1":"hatchback","a3 sedan":"saloon","a3":"hatchback","a4":"saloon/wagon","a5":"saloon","a6":"saloon/wagon","a7":"hatchback/hatchback sportback","a8":"saloon",
             "q2":"suv","q3":"suv/suv sportback","q5":"suv/suv sportback","q7":"suv","q8":"suv","e-tron":"suv/suv sportback"}

for audi_model,body_type in audi_series_bt.items():
    df.loc[(df["Manufacturer"]=="Audi")&(df["Model"].str.contains(audi_model)),"Body Type"]=body_type
print(f"Total Audi cars: {df[df['Manufacturer']=='Audi'].shape[0]}")
df.head()

Total Audi cars: 98


,Manufacturer,Model,Body Type,Year,Fuel Type,Seating Capacity,Ground Clearance,Price,Mileage,Average Price (0-20k km),Average Price (20k-50k km),Average Price (50k-100k km),Average Price (100k+ km),Condition
45,Audi,a1,hatchback,2018.0,Petrol,None,None,9600000,72000,NaN,NaN,9200000.0,NaN,NaN
66,Audi,a1,hatchback,2017.0,Petrol,None,None,8800000,91600,NaN,NaN,9200000.0,NaN,NaN
75,Audi,a1,hatchback,2016.0,Petrol,None,None,9350000,87000,NaN,NaN,9350000.0,NaN,NaN
46,Audi,a1 s-line rs1,hatchback,2018.0,Petrol,None,None,10890000,27000,NaN,10890000.0,NaN,NaN,NaN
76,Audi,a1 tfsi,hatchback,2016.0,Petrol,None,None,9700000,43000,NaN,9700000.0,NaN,NaN,NaN


Seating capacity

In [61]:
##Seatsing Capacity 
audi_series_sc={"a1":5,"a3":5,"a4":5,"a5":5,"a6":5,"a7":5,"a8":5,
             "q2":5,"q3":5,"q5":5,"q7":7,"q8":5,"e-tron":5}

bmw_series_sc={"218i":5,"316i":5,"318i":5,"320d":5,"430i gran":5,"520d":5,
            "520i":5,"523i":5,"530e":5,"530i":5,"535i":5,"725d":5,"730d":5,"730ld":5,
            "730le":5,"740li":5,"f10":5,"active 7":5,"i3":4,"i5":5,"i7":5,"i8":4,"i8 roadster":2,
            "ix":5,"m760":5,"mini cooper":4,"x1":5,"x2":5,"x3":5,"x5":5,"x6":5,"x7":7,"z4":2}

byd_series_sc={"e6":5,"seal":5,"sealion":5,"atto":5}

for audi_model,seats in audi_series_sc.items():
    df.loc[(df["Manufacturer"]=="Audi")&(df["Model"].str.contains(audi_model)),"Seating Capacity"]=seats
print(f"Total Audi cars: {df[df['Manufacturer']=='Audi'].shape[0]}")

for bmw_model,seats in bmw_series_sc.items():
    df.loc[(df["Manufacturer"]=="BMW")&(df["Model"].str.contains(bmw_model)),"Seating Capacity"]=seats
print(f"Total BMW cars: {df[df['Manufacturer']=='BMW'].shape[0]}")

for byd_model,seats in byd_series_sc.items():
    df.loc[(df["Manufacturer"]=="BYD")&(df["Model"].str.contains(byd_model)),"Seating Capacity"]=seats

df.head()


Total Audi cars: 98
Total BMW cars: 165


,Manufacturer,Model,Body Type,Year,Fuel Type,Seating Capacity,Ground Clearance,Price,Mileage,Average Price (0-20k km),Average Price (20k-50k km),Average Price (50k-100k km),Average Price (100k+ km),Condition
45,Audi,a1,hatchback,2018.0,Petrol,5,None,9600000,72000,NaN,NaN,9200000.0,NaN,NaN
66,Audi,a1,hatchback,2017.0,Petrol,5,None,8800000,91600,NaN,NaN,9200000.0,NaN,NaN
75,Audi,a1,hatchback,2016.0,Petrol,5,None,9350000,87000,NaN,NaN,9350000.0,NaN,NaN
46,Audi,a1 s-line rs1,hatchback,2018.0,Petrol,5,None,10890000,27000,NaN,10890000.0,NaN,NaN,NaN
76,Audi,a1 tfsi,hatchback,2016.0,Petrol,5,None,9700000,43000,NaN,9700000.0,NaN,NaN,NaN


Ground clearance

In [ ]:
audi_series_gc={"a1":136,"a3":140,"a4":165,"a5":118,"a6":140,"a7":5,"a8":5,
             "q2":5,"q3":5,"q5":5,"q7":7,"q8":5,"e-tron":5}

Dropping 

In [62]:
df=df.drop_duplicates(subset=["Manufacturer","Model","Year"],keep="first")

df.head()

,Manufacturer,Model,Body Type,Year,Fuel Type,Seating Capacity,Ground Clearance,Price,Mileage,Average Price (0-20k km),Average Price (20k-50k km),Average Price (50k-100k km),Average Price (100k+ km),Condition
45,Audi,a1,hatchback,2018.0,Petrol,5,None,9600000,72000,NaN,NaN,9200000.0,NaN,NaN
66,Audi,a1,hatchback,2017.0,Petrol,5,None,8800000,91600,NaN,NaN,9200000.0,NaN,NaN
75,Audi,a1,hatchback,2016.0,Petrol,5,None,9350000,87000,NaN,NaN,9350000.0,NaN,NaN
46,Audi,a1 s-line rs1,hatchback,2018.0,Petrol,5,None,10890000,27000,NaN,10890000.0,NaN,NaN,NaN
76,Audi,a1 tfsi,hatchback,2016.0,Petrol,5,None,9700000,43000,NaN,9700000.0,NaN,NaN,NaN


In [63]:
df.to_csv("./datasets/cleaned_vehicles.csv",index=False)
print(f"Saved {len(df)} records to cleaned_vehicles.csv")

Saved 3555 records to cleaned_vehicles.csv
